<p style="text-align:center">
    <a href="https://nbviewer.jupyter.org/github/twMr7/PyTorch-Deep-Learning/blob/master/01-Basic_Tensors.ipynb">
        Open In Jupyter nbviewer
        <img style="float: center;" src="https://nbviewer.jupyter.org/static/img/nav_logo.svg" width="120" />
    </a>
</p>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/twMr7/PyTorch-Deep-Learning/blob/master/01-Basic_Tensors.ipynb)

# 1. Basic Tensors

深度學習有許多有趣的應用，處理的資料不外乎文字、數字、影像、語音…等等的資料，但不管是什麼類型的資料，最終類神經網路由輸入到輸出的過程處理的還是數字，所以本質上都是在操作一堆數字的集合。 PyTorch 使用 *tensor* 作爲處理數字集合的基礎資料結構，與 *numpy ndarray* 非常類似，*tensor* 涵蓋了向量、矩陣、以及高維度陣列的概念，並且延伸加入了許多適用於深度學習的能力，例如最佳化的自動微分運算、在GPU上加速運算、分散在不同CPU、GPU、TPU裝置節點上的平行運算等。


+ [**1.1 建構 Tensor**](#construct-tensor)
+ [**1.2 Tensor 的屬性與基本操作**](#tensor-attributes)
+ [**1.3 Tensor 的數學運算**](#math-ops)
+ [**參考資料**](#references)


<a id="construct-tensor"></a>

## 1.1 建構 Tensor

Tensor 可以透過幾種不同的方式建構出來（[torch API - Creation Ops](https://pytorch.org/docs/stable/torch.html#creation-ops)）：
- 由現成的 *array_like* 的資料結構，如 `list` 或 `numpy.array`。
- 使用內建函式，由特定數值模式安排建構新的 tensor。

In [1]:
import torch
import numpy as np

---

### § 由 *list* 建構

- [**`torch.tensor()`**](https://pytorch.org/docs/stable/generated/torch.tensor.html) - 會進行資料複製的操作。
- [**`torch.as_tensor()`**](https://pytorch.org/docs/stable/generated/torch.as_tensor.html) - 資料是 *ndarray* 時儘可能不進行複製。

未明確指定 *dtype* 的話，資料型態會從元素中自動推測。


In [2]:
# 一層的 list
list1 = [1, 2, 3]

# 使用 tensor()
tensor_list1 = torch.tensor(list1)

print(tensor_list1, '\ndata type is', tensor_list1.dtype)

tensor([1, 2, 3]) 
data type is torch.int64


In [3]:
# 兩層的 list，注意其中的元素同時有浮點數和整數
list2 = [[1., 2., 3.], [4, 5, 6], [7, 8, 9]]

# 使用 as_tensor()
tensor_list2 = torch.as_tensor(list2)

print(tensor_list2, '\ndata type is', tensor_list2.dtype)

tensor([[1., 2., 3.],
        [4., 5., 6.],
        [7., 8., 9.]]) 
data type is torch.float32


---

### § 由 *numpy ndarray* 建構

*NumPy* 畢竟還是目前 Python 界最受歡迎的多維陣列函式庫，幾乎已經是許多機器學習和科學計算函式庫的共通標準了，PyTorch 對 *tensor* 的設計自然保留了許多可以相容於 *numpy ndarray* 的介面。

- [**`torch.from_numpy()`**](https://pytorch.org/docs/stable/generated/torch.from_numpy.html) - 明確不複製，與 ndarray 共享記憶體。
- [**`torch.Tensor.numpy()`**](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.numpy) - 將 tensor 轉成 numpy ndarray。

`torch.tensor()`以及`torch.as_tensor()`一樣可以用 *ndarray* 建構 *tensor* ，但爲了避免“複製”這種昂貴的操作，建議 *ndarray* 指定使用`torch.from_numpy()`。 當 *tensor* 與 *ndarray* 共用同一份底層的記憶體時，改變 *tensor* 的內容也會同時變更 *ndarray* 的內容；同理，改變 *ndarray* 元素一樣會變更 *tensor* 的元素。


In [4]:
# 二維矩陣的 array 
array2 = np.array(list2)

# 使用 from_numpy() 生成 tensor
tensor_array2 = torch.from_numpy(array2)

print(tensor_array2)

tensor([[1., 2., 3.],
        [4., 5., 6.],
        [7., 8., 9.]], dtype=torch.float64)


In [5]:
# 將 tensor 轉爲 numpy array
t2a = tensor_array2.numpy()

print('-- tensor to array:\n', t2a)

# 內容與原始的 array 一樣
print('\n-- array2:\n', array2)

-- tensor to array:
 [[1. 2. 3.]
 [4. 5. 6.]
 [7. 8. 9.]]

-- array2:
 [[1. 2. 3.]
 [4. 5. 6.]
 [7. 8. 9.]]


In [6]:
# 底層記憶體其實是共用的
t2a[-1, -1] = 10

print('-- tensor to array:\n', t2a)

# 所有共用記憶體的內容都一起改變了
print('\n-- tensor_array2:\n', tensor_array2)
print('\n-- array2:\n', array2)

-- tensor to array:
 [[ 1.  2.  3.]
 [ 4.  5.  6.]
 [ 7.  8. 10.]]

-- tensor_array2:
 tensor([[ 1.,  2.,  3.],
        [ 4.,  5.,  6.],
        [ 7.,  8., 10.]], dtype=torch.float64)

-- array2:
 [[ 1.  2.  3.]
 [ 4.  5.  6.]
 [ 7.  8. 10.]]


---

### § 由內建函式建構新的 Tensor

內建用來建構 *tensor* 的函式，有很多名字刻意設計與 *numpy* 的一樣，讓原本熟悉 *numpy* 的開發者可以很快上手。
- [**`torch.zeros()`**](https://pytorch.org/docs/stable/generated/torch.zeros.html) - 生成元素都是 **0** 的 *tensor*。
- [**`torch.ones()`**](https://pytorch.org/docs/stable/generated/torch.ones.html) - 生成元素都是 **1** 的 *tensor*。
- [**`torch.zeros_like()`**](https://pytorch.org/docs/stable/generated/torch.zeros_like.html) - 比照輸入物件的維度及大小，生成元素都是 **0** 的 *tensor*。
- [**`torch.ones_like()`**](https://pytorch.org/docs/stable/generated/torch.ones_like.html) - 比照輸入物件的維度及大小，生成元素都是 **1** 的 *tensor*。
- [**`torch.arange()`**](https://pytorch.org/docs/stable/generated/torch.arange.html) - 用指定範圍的數列（通常是整數）產生一維 *tensor*。
- [**`torch.linspace()`**](https://pytorch.org/docs/stable/generated/torch.arange.html) - 在指定上下限數字間（通常是浮點數），以均勻間隔的數列產生一維 *tensor*。
- [**`torch.rand()`**](https://pytorch.org/docs/stable/generated/torch.rand.html) - 從 *uniform* 分佈裡產生 $[0,1)$ 之間指定維度及大小的隨機數 *tensor*。
- [**`torch.randn()`**](https://pytorch.org/docs/stable/generated/torch.randn.html) - 從 *standard normal* 分佈（平均爲0且變異數爲1）裡產生之間指定維度及大小的隨機數 *tensor*。


In [7]:
# 比照 tensor_array2 的形狀，產生一個新的元素都爲0的 tensor
tensor_zero2 = torch.zeros_like(tensor_array2)

print('-- tensor zero2:\n', tensor_zero2)

-- tensor zero2:
 tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]], dtype=torch.float64)


In [8]:
# 指定 5x5 的尺寸，產生標準常態分佈隨機數
tensor_randn = torch.randn((5,5))

print('-- tensor 5x5 randn:\n', tensor_randn)

-- tensor 5x5 randn:
 tensor([[ 0.1132,  0.1963,  0.6799, -1.6679,  1.2001],
        [ 0.4269,  0.7143, -0.4991,  0.9837, -1.3789],
        [ 0.8069, -1.1253, -1.0976, -0.7707,  0.1946],
        [-0.8130,  2.3870, -0.2506, -0.3445, -0.7529],
        [-0.5718, -1.5279,  0.9815,  1.3052,  0.6815]])


<a id="tensor-attributes"></a>

## 1.2 Tensor 的屬性與基本操作

[**Tensor 的屬性**](https://pytorch.org/docs/stable/tensor_attributes.html)

每一個 *tensor* 物件都包含了描述 *tensor* 的形狀、大小、資料型態等屬性。
- [**`torch.dtype`**](https://pytorch.org/docs/stable/tensor_attributes.html#torch.torch.dtype) - 資料型態。
- [**`torch.shape`**](https://pytorch.org/docs/stable/tensor_attributes.html#torch.torch.dtype) - 維度形狀，`size()`的別名。
- [**`torch.size()`**](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.size) - 維度形狀。
- [**`torch.dim()`**](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.dim) - 維度。
- [**`torch.numel()`**](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.size) - 元素個數。
- [**`torch.device`**](https://pytorch.org/docs/stable/tensor_attributes.html#torch.torch.device) - 資料配置所在的裝置。
- [**`torch.Tensor.stride()`**](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.stride) - 每個維度成員之間的步幅、間距。
- [**`torch.Tensor.storage()`**](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.storage) - 底層的儲存物件。
- [**`torch.Tensor.storage_type()`**](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.storage_type) - 底層的儲存型態。


In [27]:
print('dtype:', tensor_randn.dtype)
print('shape:', tensor_randn.shape)
print('size:', tensor_randn.size())
print('numel:', tensor_randn.numel())
print('device:', tensor_randn.device)
print('stride:', tensor_randn.stride())
print('storage type:', tensor_randn.storage_type())
print('storage:', tensor_randn.storage())

dtype: torch.float32
shape: torch.Size([5, 5])
size: torch.Size([5, 5])
numel: 25
device: cpu
stride: (5, 1)
storage type: <class 'torch.FloatStorage'>
storage:  0.11315546929836273
 0.1962723433971405
 0.6798641681671143
 -1.6679118871688843
 1.2000519037246704
 0.426929771900177
 0.7142747640609741
 -0.49911901354789734
 0.9836630821228027
 -1.3789368867874146
 0.8068822622299194
 -1.1252902746200562
 -1.0975691080093384
 -0.7707319259643555
 0.1946474015712738
 -0.8129519820213318
 2.3870155811309814
 -0.25057148933410645
 -0.3444853723049164
 -0.7529102563858032
 -0.5718392729759216
 -1.5279439687728882
 0.9815207719802856
 1.305169939994812
 0.6814917325973511
[torch.FloatStorage of size 25]


---

### § Tensor 的資料型態及轉換方法

建構 *tensor* 的函式都可以直接指定配置那一種資料型態，如 `torch.float`、`torch.double`、`torch.int`、 ...等等，或是在需要時進行轉換。

- [**`tensor.to()`**](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.to) - 轉換資料型態或暫存記憶體裝置的通用函式。
- [**`tensor.double()`**](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.double) - 轉換資料型態爲 *double* (64-bit 浮點數)。
- [**`tensor.float()`**](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.float) - 轉換資料型態爲 *float* (32-bit 浮點數)。
- [**`tensor.half()`**](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.half) - 轉換資料型態爲 *half* (16-bit 浮點數)。
- [**`tensor.int()`**](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.int) - 轉換資料型態爲 *int* (32-bit 整數)。
- ...

In [10]:
# 建構指定範圍的數列
tensor_arange = torch.arange(2, 7)

print('-- tensor arange:\n', tensor_arange, ', dtype:', tensor_arange.dtype)

-- tensor arange:
 tensor([2, 3, 4, 5, 6]) , dtype: torch.int64


In [11]:
# 轉換資料型態，使用 half()
tensor_halfarange = tensor_arange.half()

print('-- tensor halfarange:\n', tensor_halfarange)

-- tensor halfarange:
 tensor([2., 3., 4., 5., 6.], dtype=torch.float16)


In [12]:
# 轉換資料型態，使用 to()
tensor_floatarange = tensor_halfarange.to(torch.float32)

print('-- tensor floatarange:\n', tensor_floatarange, ', dtype:', tensor_floatarange.dtype)

-- tensor floatarange:
 tensor([2., 3., 4., 5., 6.]) , dtype: torch.float32


---

### § Tensor 的計算裝置

深度學習經常需要藉由 GPU 加快訓練或推論的速度，因此 *tensor* 帶有計算的記憶體裝置屬性，可以視需求指定在 CPU 或 GPU 上計算。 *tensor* 的建構函式都有提供 *device* 的參數，未指定的話預設裝置是 CPU，`tensor.to()` 函式可以用來將資料複製到指定裝置。

- [**`tensor.to()`**](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.to) - 轉換資料型態或儲存裝置的通用函式，*device* 參數可以是字串 `'cuda'` 或 `'cpu'`。
- [**`tensor.cuda()`**](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.cuda) - 將 *tensor* 資料複製到 GPU。
- [**`tensor.cpu()`**](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.cpu) - 將 *tensor* 資料複製到 CPU。


In [25]:
# 生成 3x4 均勻分佈的隨機數 tensor
tensor_rand = torch.rand((3,4))

print('-- tensor 3x4 rand:\n{}\nDevice: {}\nStorage type: {}'.format(
    tensor_rand, tensor_rand.device, tensor_rand.storage_type()))

-- tensor 3x4 rand:
tensor([[0.8619, 0.3442, 0.2436, 0.6928],
        [0.0634, 0.1429, 0.0647, 0.5768],
        [0.3780, 0.8026, 0.2909, 0.3807]])
Device: cpu
Storage type: <class 'torch.FloatStorage'>


In [26]:
# 檢查 GPU 裝置是否存在
if torch.cuda.is_available():
    tensor_randgpu = tensor_rand.to('cuda')
    print('-- tensor rand on GPU:\n{}\nDevice: {}\nStorage type: {}'.format(
        tensor_randgpu, tensor_randgpu.device, tensor_randgpu.storage_type()))
else:
    print('GPU device is not available.')

-- tensor rand on GPU:
tensor([[0.8619, 0.3442, 0.2436, 0.6928],
        [0.0634, 0.1429, 0.0647, 0.5768],
        [0.3780, 0.8026, 0.2909, 0.3807]], device='cuda:0')
Device: cuda:0
Storage type: <class 'torch.cuda.FloatStorage'>


---

### § Tensor 的索引方法

*Tensor* 的索引方法與 *numpy ndarray* 以及 python 內建的 *list* 相同。

In [15]:
print(tensor_array2[:])
print(tensor_array2[-1])
print(tensor_array2[1:-1])

tensor([[ 1.,  2.,  3.],
        [ 4.,  5.,  6.],
        [ 7.,  8., 10.]], dtype=torch.float64)
tensor([ 7.,  8., 10.], dtype=torch.float64)
tensor([[4., 5., 6.]], dtype=torch.float64)


In [16]:
print(tensor_array2[1, :])
print(tensor_array2[0:2, 2])
print(tensor_array2[:-1, :-1])

tensor([4., 5., 6.], dtype=torch.float64)
tensor([3., 6.], dtype=torch.float64)
tensor([[1., 2.],
        [4., 5.]], dtype=torch.float64)


---

### § Tensor 的串接分割與維度調整

- [**`torch.cat()`**](https://pytorch.org/docs/stable/generated/torch.cat.html) - 按照指定的維度方向串接多個 *tensor*。
- [**`torch.split()`**](https://pytorch.org/docs/stable/generated/torch.split.html) - 將 *tensor* 按照指定的方式切割成小塊。
- [**`torch.reshape()`**](https://pytorch.org/docs/stable/generated/torch.reshape.html) - 將 *tensor* 調整成指定的維度形狀，若形狀相容返回 *view*，否則會複製。
- [**`torch.transpose()`**](https://pytorch.org/docs/stable/generated/torch.transpose.html) - 轉置 *tensor* 指定的維度。
- [**`torch.t()`**](https://pytorch.org/docs/stable/generated/torch.t.html) - 一般二維矩陣的轉置，等同 `transpose(input, 0, 1)`。
- ...

許多 *tensor* 的操作支援共享現有記憶體中數據的 [**view**](https://pytorch.org/docs/stable/tensor_view.html)，這些操作避免了記憶體複製，可以更快更有效率地存取片段、調整形狀等。

- [**`torch.Tensor.view()`**](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.view) - 返回指定的維度形狀的 *view* 。
- [**`torch.Tensor.squeeze()`**](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.squeeze) - 返回沒有任何維度是 1 的 *view*。
- [**`torch.Tensor.unsqueeze()`**](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.unsqueeze) - 返回插入 1 個指定維度的 *view*。
- [**`torch.Tensor.split()`**](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.split) - 返回指定方式切割成小塊的 *view*。
- [**`torch.Tensor.transpose()`**](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.transpose) - 返回指定維度置換後的 *view*。
- [**`torch.Tensor.t()`**](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.t) - 一般二維矩陣的轉置，等同 `transpose(input, 0, 1)`。
- [**`torch.Tensor.T`**](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.t) - 返回所有維度反向的 *view*，如果是二維矩陣，效果與 `torch.Tensor.t()` 相同。
- ...

***Note***: 有些操作是定義在 *torch* 的方法，如 **`torch.cat()`**；有些是定義在 *torch.Tensor* 的方法，如 **`torch.Tensor.view()`**；有些則是呼叫在 *torch* 定義的或是 *torch.Tensor* 的都一樣，例如 **`torch.split()`** 與 **`torch.Tensor.split()`**。

***Note***: 看到 **torch.Tensor** 的方法名字後面有底線的，就是對應相同方法的 in-place（就地變更）版本。 如 `torch.Tensor.transpose_()` 就是對應 `torch.Tensor.transpose()` 的就地變更版本。


In [17]:
# concatenate a sequence of tensors
wide_cat = torch.cat([tensor_array2, tensor_zero2, tensor_rand], dim=1)

print('A sequence of tensors concatenated, shape={}:\n{}'.format(wide_cat.shape, wide_cat))

A sequence of tensors concatenated, shape=torch.Size([3, 10]):
tensor([[ 1.0000,  2.0000,  3.0000,  0.0000,  0.0000,  0.0000,  0.7083,  0.5107,
          0.0899,  0.2314],
        [ 4.0000,  5.0000,  6.0000,  0.0000,  0.0000,  0.0000,  0.3705,  0.1365,
          0.1549,  0.0774],
        [ 7.0000,  8.0000, 10.0000,  0.0000,  0.0000,  0.0000,  0.7773,  0.7199,
          0.4946,  0.8288]], dtype=torch.float64)


In [18]:
# view with a different shape
long_view = wide_cat.view(15, -1)

print('A long view from wide:\n', long_view)

A long view from wide:
 tensor([[ 1.0000,  2.0000],
        [ 3.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.7083,  0.5107],
        [ 0.0899,  0.2314],
        [ 4.0000,  5.0000],
        [ 6.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.3705,  0.1365],
        [ 0.1549,  0.0774],
        [ 7.0000,  8.0000],
        [10.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.7773,  0.7199],
        [ 0.4946,  0.8288]], dtype=torch.float64)


In [20]:
# 底層儲存實際數據的記憶體是一樣的位置
long_view.storage().data_ptr() == wide_cat.storage().data_ptr()

True

In [23]:
print('long_view stride = {}'.format(long_view.stride()))
print('wide_cat stride = {}'.format(wide_cat.stride()))

long_view stride = (2, 1)
wide_cat stride = (10, 1)


In [21]:
# column 方向分割固定長度
for s in long_view.split(5, dim=0):
    print(s)

tensor([[1.0000, 2.0000],
        [3.0000, 0.0000],
        [0.0000, 0.0000],
        [0.7083, 0.5107],
        [0.0899, 0.2314]], dtype=torch.float64)
tensor([[4.0000, 5.0000],
        [6.0000, 0.0000],
        [0.0000, 0.0000],
        [0.3705, 0.1365],
        [0.1549, 0.0774]], dtype=torch.float64)
tensor([[ 7.0000,  8.0000],
        [10.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.7773,  0.7199],
        [ 0.4946,  0.8288]], dtype=torch.float64)


In [22]:
# column 方向分割指定長度
for s in long_view.split([3, 5, 7], dim=0):
    print(s)

tensor([[1., 2.],
        [3., 0.],
        [0., 0.]], dtype=torch.float64)
tensor([[0.7083, 0.5107],
        [0.0899, 0.2314],
        [4.0000, 5.0000],
        [6.0000, 0.0000],
        [0.0000, 0.0000]], dtype=torch.float64)
tensor([[ 0.3705,  0.1365],
        [ 0.1549,  0.0774],
        [ 7.0000,  8.0000],
        [10.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.7773,  0.7199],
        [ 0.4946,  0.8288]], dtype=torch.float64)


<a id="math-ops"></a>

## 1.3 Tensor 的數學運算

[**數學運算**](https://pytorch.org/docs/stable/torch.html#math-operations)

算數運算子 " $+\,-\,*\,/$ " 的操作與 *numpy* 有相同的效果，矩陣乘法運算子 $@$ 也一樣，運算的 *broadcast* 概念也相同。

- [**`torch.square()`**](https://pytorch.org/docs/stable/generated/torch.square.html) - 平方。
- [**`torch.exp()`**](https://pytorch.org/docs/stable/generated/torch.exp.html) - exponential。
- [**`torch.log()`**](https://pytorch.org/docs/stable/generated/torch.log.html) - natural logarithm。
- [**`torch.sum()`**](https://pytorch.org/docs/stable/generated/torch.sum.html) - 所有元素的加總。
- [**`torch.mean()`**](https://pytorch.org/docs/stable/generated/torch.mean.html) - 所有元素的平均。
- [**`torch.std()`**](https://pytorch.org/docs/stable/generated/torch.std.html) - 所有元素的標準差。
- ...

運算結果若是單一元素的純量，仍然會是返回 *tensor* 型態的資料，可以透過 [**`torch.Tensor.item()`**](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.item) 取出標準的 python 純量數值型別。 否則，多個元素的 *tensor* 也可以透過 [**`torch.Tensor.tolist()`**](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.tolist) 轉成 python 的 *list* 型別資料。

In [31]:
# 現有變數
print(tensor_array2)
print(tensor_rand)

tensor([[ 1.,  2.,  3.],
        [ 4.,  5.,  6.],
        [ 7.,  8., 10.]], dtype=torch.float64)
tensor([[0.8619, 0.3442, 0.2436, 0.6928],
        [0.0634, 0.1429, 0.0647, 0.5768],
        [0.3780, 0.8026, 0.2909, 0.3807]])


In [30]:
# element-wise 一對一相乘
tensor_array2 * tensor_array2

tensor([[  1.,   4.,   9.],
        [ 16.,  25.,  36.],
        [ 49.,  64., 100.]], dtype=torch.float64)

In [33]:
# element-wise 平方
torch.square(tensor_array2)

tensor([[  1.,   4.,   9.],
        [ 16.,  25.,  36.],
        [ 49.,  64., 100.]], dtype=torch.float64)

In [32]:
# 乘法、加法、與 broadcast，使用算數運算子
tensor_array2 * 2 + tensor_rand[:,0]

tensor([[ 2.8619,  4.0634,  6.3780],
        [ 8.8619, 10.0634, 12.3780],
        [14.8619, 16.0634, 20.3780]], dtype=torch.float64)

In [35]:
# 乘法、加法、與 broadcast，使用函式呼叫
torch.add(torch.multiply(tensor_array2, 2), tensor_rand[:,0])

tensor([[ 2.8619,  4.0634,  6.3780],
        [ 8.8619, 10.0634, 12.3780],
        [14.8619, 16.0634, 20.3780]], dtype=torch.float64)

In [28]:
# 矩陣乘法，使用算數運算子
tensor_array2 @ tensor_array2.T

tensor([[ 14.,  32.,  53.],
        [ 32.,  77., 128.],
        [ 53., 128., 213.]], dtype=torch.float64)

In [36]:
# 矩陣乘法，使用函式呼叫
torch.matmul(tensor_array2, tensor_array2.T)

tensor([[ 14.,  32.,  53.],
        [ 32.,  77., 128.],
        [ 53., 128., 213.]], dtype=torch.float64)

In [38]:
# 計算結果為單一元素
torch.sum(tensor_rand) / tensor_rand.numel()

tensor(0.4036)

In [37]:
# 單一元素取 python 數值
torch.mean(tensor_rand).item()

0.4035583436489105

<a id="references"></a>

## References:
+ PyTorch Documentation [[link]](https://pytorch.org/docs/)
+ Eli Stevens, Luca Antiga, and Thomas Viehmann. "*Deep Learning with PyTorch*". Manning Publications Co, 2020. [[link]](https://pytorch.org/assets/deep-learning/Deep-Learning-with-PyTorch.pdf)
+ Suraj Subramanian, et al. "*Learn the Basics - Tensors*". PyTorch.org. [[link]](https://pytorch.org/tutorials/beginner/basics/tensor_tutorial.html)


---

**May, 2021**

***James Chang***